<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_time_series/class_01/TS%20-%20W1%20-%2008%20-%20Previs%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Previsão estatística em dados sintéticos

Neste laboratório, você fará algumas previsões estatísticas para poder compará-las com os modelos de aprendizado de máquina que serão criados posteriormente.

## Importações

Primeiro, você importará os pacotes de que precisará para executar todo o código deste laboratório. Você usará:
* [Tensorflow](https://www.tensorflow.org/api_docs/python/tf) para criar seu modelo e preparar janelas de dados
* [Numpy](https://numpy.org/) para processamento numérico
* e a biblioteca [PyPlot](https://matplotlib.org/3.5.1/api/_as_gen/matplotlib.pyplot.html) do Matplotlib para visualização

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Utilitários

Em seguida, você definirá algumas funções utilitárias para tornar o código mais organizado. A primeira é a função de plotagem que você também usou no laboratório anterior.

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    """
    Visualiza dados de séries temporais

    Args:
      time (array of int) - contém as etapas de tempo
      series (array of int) - contém as medidas para cada etapa de tempo
      format - estilo da linha ao plotar o gráfico
      label - rótulo da linha
      start - primeiro passo de tempo a ser plotado
      end - última etapa de tempo a ser plotada
    """
    # Configuração das dimensões da figura do gráfico
    plt.figure(figsize=(10, 6))

    if type(series) is tuple:

      for series_num in series:
        # Plotar os dados da série temporal
        plt.plot(time[start:end], series_num[start:end], format)

    else:
      # Plotar os dados da série temporal
      plt.plot(time[start:end], series[start:end], format)

    # Rotular o eixo x
    plt.xlabel("Time")

    # # Rotular o eixo y
    plt.ylabel("Value")

    # Sobrepor uma grade no gráfico
    plt.grid(True)

    # Desenhe o gráfico na tela
    plt.show()

Você também colocará aqui as funções para gerar seus dados sintéticos.

Para este laboratório, você só precisará de tendência, sazonalidade e ruído.

Sinta-se à vontade para adicionar outros elementos posteriormente, caso queira uma tarefa mais desafiadora.

In [ ]:
def trend(time, slope=0):
    """
    Visualiza dados de séries temporais

    Args:
      time (vetor de int) - contém as etapas de tempo
      series (vetor de int) - contém as medidas para cada etapa de tempo
      format - estilo da linha ao plotar o gráfico
      label - rótulo da linha
      start - primeiro passo de tempo a ser plotado
      end - última etapa de tempo a ser plotada
    """

    # Calcule a série linear dada a inclinação
    series = slope * time

    return series

def seasonal_pattern(season_time):
    """
    Apenas um padrão arbitrário, você pode alterá-lo se desejar

    Args:
      season_time (vetor de float) - contém as medições por etapa de tempo

    Retorna:
      data_pattern (vetor de float) - contém os valores de medição revisados de acordo com
                                  de acordo com o padrão definido
    """

    # Gerar os valores usando um padrão arbitrário
    data_pattern = np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

    return data_pattern

def seasonality(time, period, amplitude=1, phase=0):
    """
    Repete o mesmo padrão em cada período

    Args:
      time (vetor de int) - contém as etapas de tempo
      period (int) - número de etapas de tempo antes da repetição do padrão
      amplitude (int) - valor de pico medido em um período
      phase (int) - número de etapas de tempo para deslocar os valores medidos

    Retorna:
      data_pattern (vetor de float) - dados sazonais dimensionados pela amplitude definida
    """

    # Definir os valores medidos por período
    season_time = ((time + phase) % period) / period

    # Gera os dados sazonais dimensionados pela amplitude definida
    data_pattern = amplitude * seasonal_pattern(season_time)

    return data_pattern

def noise(time, noise_level=1, seed=None):
    """Gera um sinal ruidoso normalmente distribuído

    Args:
      time (vetor de int) - contém as etapas de tempo
      noise_level (float) - fator de escala para o sinal gerado
      seed (int) - semente do gerador de números para repetibilidade

    Retorna:
      noise (vetor de float) - o sinal ruidoso
    """

    # Inicializar o gerador de números aleatórios
    rnd = np.random.RandomState(seed)

    # Gerar um número aleatório para cada etapa de tempo e dimensionar pelo nível de ruído
    noise = rnd.randn(len(time)) * noise_level

    return noise

## Gerar os dados sintéticos

Em seguida, você pode usar as funções utilitárias acima para gerar os dados sintéticos.

Eles começarão em uma linha de base e, em seguida, apresentarão uma tendência ascendente com um padrão sazonal a cada 365 etapas.

Você também adicionará algum ruído porque os dados do mundo real também costumam ser ruidosos.

In [ ]:
# Parâmetros
time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
amplitude = 40
slope = 0.05
noise_level = 5

# Criar a série
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)

# Atualização com ruído
series += noise(time, noise_level, seed=42)

# Plotar os resultados
plot_series(time, series)

## Dividir o conjunto de dados

Em seguida, você dividirá os dados acima em conjuntos de treinamento e validação.

Você pegará os primeiros 1.000 pontos para treinamento, enquanto o restante será para validação.

In [ ]:
# Definir o tempo de divisão
split_time = 1000

# Obter o conjunto de treino
time_train = time[:split_time]
x_train = series[:split_time]

# Obter o conjunto de validação
time_valid = time[split_time:]
x_valid = series[split_time:]

Você pode inspecionar esses conjuntos visualmente usando a mesma função de utilidade para plotagem.

Observe que, em geral, o conjunto de validação tem valores mais altos (ou seja, no eixo y) do que os do conjunto de treinamento.

Seu modelo deve ser capaz de prever esses valores apenas aprendendo com a tendência e a sazonalidade do conjunto de treinamento.

In [ ]:
# Traçar o conjunto de treino
plot_series(time_train, x_train)

In [ ]:
# Plotar o conjunto de validação
plot_series(time_valid, x_valid)

# Previsão ingênua(_Naive Forecast_)

Como linha de base, é possível fazer uma previsão ingênua em que se supõe que o próximo valor será o mesmo da etapa anterior.

Você pode cortar a série original como abaixo e imprimir alguns valores como uma verificação de sanidade.

O valor da próxima etapa de tempo deve ser idêntico ao da verdade básica na etapa de tempo anterior.

In [ ]:
# Gerar a previsão ingênua
naive_forecast = series[split_time - 1:-1]

# Definir etapa de tempo
time_step = 100

# Imprimir valores
print(f'ground truth no passo de tempo {time_step}: {x_valid[time_step]}')
print(f'predição no passo de tempo {time_step + 1}: {naive_forecast[time_step + 1]}')


Você pode ver isso visualmente com a função `plot_series()` que você definiu anteriormente.

In [ ]:
# Plotar os resultados
plot_series(time_valid, (x_valid, naive_forecast))

Você pode ampliar o zoom no início do período de validação para ver que a previsão ingênua está 1 passo atrás da série temporal.

In [ ]:
# Aumentar o zoom
plot_series(time_valid, (x_valid, naive_forecast), start=0, end=150)

### Computação de métricas

Agora você calculará o [erro quadrático médio](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/mean_squared_error) e o [erro absoluto médio](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/mean_absolute_error) entre as previsões e as predições no período de validação. Eles estão disponíveis por meio da API [`tf.keras.metrics`](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/).

In [ ]:
print(tf.keras.metrics.mean_squared_error(x_valid, naive_forecast).numpy())
print(tf.keras.metrics.mean_absolute_error(x_valid, naive_forecast).numpy())

Os valores acima serão sua linha de base e você verá se pode usar outros métodos para obter um desempenho melhor do que a previsão ingênua.

## Média móvel

Uma técnica que você pode usar é fazer uma média móvel. Ela soma uma série de etapas de tempo e a média será a previsão para a próxima etapa de tempo.
> Por exemplo, a média das medições nas etapas de tempo 1 a 10 será a previsão para a etapa de tempo 11, depois a média das etapas de tempo 2 a 11 será a previsão para a etapa de tempo 12 e assim por diante.

A função abaixo faz a média móvel para toda a "série". Ela recebe um argumento `window_size` para indicar o número de etapas de tempo a serem consideradas no cálculo da média.

In [ ]:
def moving_average_forecast(series, window_size):
    """Gera uma previsão de média móvel

    Args:
      series (array of float) - contém os valores da série temporal
      window_size (int) - o número de etapas de tempo para calcular a média

    Retorna:
      forecast (array of float) - a previsão da média móvel
    """

    # Inicializar uma lista
    forecast = []

    # Calcular a média móvel com base no tamanho da janela
    for time in range(len(series) - window_size):
      forecast.append(series[time:time + window_size].mean())

    # Converta em uma matriz numérica
    forecast = np.array(forecast)

    return forecast

Você usará essa função para gerar a previsão com um tamanho de janela de `30`.

In [ ]:
# Gerar a previsão de média móvel
moving_avg = moving_average_forecast(series, 30)[split_time - 30:]

# Plotar os resultados
plot_series(time_valid, (x_valid, moving_avg))

In [ ]:
# Compute the metrics
print(tf.keras.metrics.mean_squared_error(x_valid, moving_avg).numpy())
print(tf.keras.metrics.mean_absolute_error(x_valid, moving_avg).numpy())

Isso é pior do que uma previsão ingênua!
> A média móvel não prevê tendência ou sazonalidade.

Em particular, esses picos enormes na série original causam grandes desvios, conforme mostrado no gráfico acima.

Você tentará remover essas características do conjunto de dados com a diferenciação de tempo e verá se obtém melhores resultados.

## Diferenciação

Como o período de sazonalidade é de 365 dias, você subtrairá o valor no momento *t* - 365 do valor no momento *t*. Isso é feito com o código abaixo.

Além disso, você precisará alinhar o resultado com a matriz `time`. Como só é possível fazer a diferenciação de tempo para `t >= 365`, você precisará truncar as primeiras 365 etapas de tempo da matriz `time`.

Você pode plotar o resultado para visualizar os valores.

In [ ]:
# Subtraia os valores em t-365 da série original
diff_series = (series[365:] - series[:-365])

# Truncar as primeiras 365 etapas de tempo
diff_time = time[365:]

# Plotar os resultados
plot_series(diff_time, diff_series)

Excelente! A tendência e a sazonalidade parecem ter desaparecido, portanto, agora você pode tentar novamente usando a média móvel.

`diff_series` é o _ground-truth_, enquanto `diff_moving_avg` é a matriz de previsão.

Você os dividirá de acordo para corresponder às etapas de tempo do conjunto de validação antes de compará-los.

In [ ]:
# Gerar média móvel a partir do conjunto de dados com diferença de tempo
diff_moving_avg = moving_average_forecast(diff_series, 30)

# Corte os pontos de previsão que correspondem às etapas de tempo do conjunto de validação
diff_moving_avg = diff_moving_avg[split_time - 365 - 30:]

# Corte os pontos da verdade básica que correspondem às etapas de tempo do conjunto de validação
diff_series = diff_series[split_time - 365:]

# Plotar os resultados
plot_series(time_valid, (diff_series, diff_moving_avg))

Agora você trará de volta a tendência e a sazonalidade adicionando os valores anteriores de `t - 365`:

In [ ]:
# Adicione a tendência e a sazonalidade da série original
diff_moving_avg_plus_past = series[split_time - 365:-365] + diff_moving_avg

# Plotar os resultados
plot_series(time_valid, (x_valid, diff_moving_avg_plus_past))

In [ ]:
print(tf.keras.metrics.mean_squared_error(x_valid, diff_moving_avg_plus_past).numpy())
print(tf.keras.metrics.mean_absolute_error(x_valid, diff_moving_avg_plus_past).numpy())

É um pouco melhor do que a previsão ingênua.

Entretanto, as previsões parecem um pouco aleatórias demais porque você está adicionando valores passados que já são ruidosos.

Lembre-se de que o sinal diferenciado no tempo também é ruidoso, portanto, adicionar esses valores passados brutos pode agravar esse problema.

Para remediar isso, você pode usar uma média móvel nos valores passados para suavizar parte desse ruído.

## Suavização

Você pode usar a mesma função `moving_average_forecast()` para suavizar os valores passados antes de adicioná-los novamente à média móvel com diferença de tempo. Há duas maneiras de fazer isso:

* Janelas finais - Isso se refere à obtenção da média dos valores anteriores para suavizar o valor na etapa de tempo atual. Por exemplo, obter a média de `t=0` a `t=6` para obter o ponto de dados suavizado em **`t=6`**.

* Janelas centralizadas - Isso se refere à obtenção da média dos valores passados e futuros para suavizar o valor na etapa de tempo atual. Por exemplo, obter a média de `t=0` a `t=6` para obter o ponto de dados suavizado em **`t=3`**.

O código abaixo usará a abordagem de janelas centralizadas e você perceberá isso no corte da matriz `series`. Ela é deslocada em `370` passos e o tamanho da janela é `11`. Para obter o ponto de dados suave em `t=1000` (ou seja, o início do conjunto de validação), ele calculará a média das medições de `t=995` a `t=1005`.

In [ ]:
# Suavize a série original antes de adicionar a média móvel com diferença de tempo
diff_moving_avg_plus_smooth_past = moving_average_forecast(series[split_time - 370:-359], 11) + diff_moving_avg

# Plotar os resultados
plot_series(time_valid, (x_valid, diff_moving_avg_plus_smooth_past))

As métricas mostrarão uma grande melhoria em relação ao resultado anterior.

In [ ]:
 # Compute the metrics
print(tf.keras.metrics.mean_squared_error(x_valid, diff_moving_avg_plus_smooth_past).numpy())
print(tf.keras.metrics.mean_absolute_error(x_valid, diff_moving_avg_plus_smooth_past).numpy())

## Resumo

Isso conclui uma breve exploração dos métodos estatísticos para previsão de séries temporais. Nos próximos laboratórios, você criará redes neurais para previsão e verá se obtém resultados comparáveis às técnicas que acabou de usar neste laboratório.